- 保监会 八分类模型 1 预处理

# 基本设置

In [2]:
import jieba
import sys
import re
import time
import string

%matplotlib inline
import numpy as np
import pandas as pd
import pre_cor
import os
import json
import urllib

import warnings
warnings.filterwarnings('ignore')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.869 seconds.
Prefix dict has been built succesfully.


# 导入数据

In [6]:
folder = '20180723' # 数据文件夹
file_list = ['20180723_circ_class_1.xlsx',]

## title + url

In [11]:
cor_data_raw = pd.DataFrame()
for file_name in file_list:
    tmp_data = pd.read_excel('data/{0}/{1}'.format(folder, file_name))
    print(tmp_data.shape, file_name)
    print()
    
    cor_data_raw = pd.concat([cor_data_raw, tmp_data], axis = 0)
    
print('总数：', cor_data_raw.shape)
cor_data_raw.head()

(3001, 5) 20180723_circ_class_1.xlsx

总数： (3001, 5)


,label,title,url,publishtime,name
0,1,做好规划不当月光族不做守财奴,https://xueqiu.com/1983578372/98551727,2018-01-02 15:11:33,噪音
1,2,做到以下三点，汽车保费想不便宜都难,http://mp.weixin.qq.com/s?__biz=MzA4ODYwMTczNg...,2018-01-04 06:59:56,噪音
2,3,做车险代理的前景如何 谊信合让你的财富事业迈上新高度,http://jxhb.91cy.cn/119134.shtml,2018-01-01 12:02:45,噪音
3,4,做保险想要高业绩，那就每天晚上8:30，来抢投保意向客户啊！,http://mp.weixin.qq.com/s?__biz=MzI2NjUyMTUxNQ...,2018-01-01 15:06:10,噪音
4,5,坐顺风车出事了，车险理赔吗,http://baijiahao.baidu.com/s?id=15887496924060...,2018-01-06 14:47:41,噪音


## content

In [ ]:
if 1:    
    raw_data = []
    for index in cor_data_raw.index:
        if index % 1000 == 0:print('index: ', index)
        try :
            label = cor_data_raw.loc[index, 'name']
            title = cor_data_raw.loc[index, 'title']
            row = cor_data_raw.loc[index, 'url']
            url = "http://121.42.224.166/extract?url=" + row +"?j=1"
            #print(url)
            request = urllib.request.Request(url)
            response = urllib.request.urlopen(request)
            pageCode = response.read().decode('utf-8')
            content = pageCode[pageCode.index("Content") + 8:]

            raw_data.append([label, title, content])
        except Exception as e:
            print('---------------')
            print(index, label, title)
            print(row)
            print(e)
            continue

    raw_data = pd.DataFrame(raw_data, columns = ['label', 'title', 'content'])
    filename = 'circ_class_combined_20180723.xlsx'
    raw_data.to_excel('data/%s'%filename, index = False)
    print(raw_data.shape)
    raw_data.head()

---------------
171 产品销售 众安平安强强联手——你的车险“保骉”正驶来
http://www.toutiao.com/i6507825874241323524
'utf-8' codec can't decode byte 0x9c in position 86: invalid start byte
---------------
541 行业 野村：第三季友邦新业务保费增长减慢 吁“买入”新华保险等
http://www.lianlianmoney.com/c/article-4-565352-0.html
HTTP Error 504: Gateway Timeout
---------------
565 噪音 延平：儿子酒驾撞坏车 父亲“顶包”骗保被查获
东南网讯（通讯员 陈宇）儿子酒驾把车撞坏了，父亲不仅没有做好教育引导，反而为骗取2万元保费自己“顶包”，企图瞒天过海。12月21日，经延平警方细致调查，该起骗保案件成功告破。http://t.cn/RHpSTjo ​
http://weibo.com/2216651507/FCjRsgnQj
HTTP Error 504: Gateway Timeout
---------------
924 监管 深圳保监局：推动保险纠纷“诉调对接”
http://www.heimaba.com/money/insurance/2018/0101/1493624.html
HTTP Error 504: Gateway Timeout
---------------
1142 产品销售 农行百色分行元旦节代理保险业务营销“开门红”
http://3g.k.sohu.com/t/n253400849
[WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。
---------------
1351 经营管理 揭秘保险业内幕：为什么保险公司一直在招人？_小大多少_mail63491821的和讯博客 ​
http://weibo.com/1305295745/FCNO0vboo
HTTP Error 504: Gateway Timeout
---------------
1457 经营管理 黄河财险获保监会批复，成为2018年首家获批开业的保险公司
ht

## label + title + content

In [ ]:
filename = 'circ_class_combined_20180723.xlsx'
raw_data = pd.read_excel('data/%s'%filename)
print(raw_data.shape)
raw_data.head()

# 预处理数据

In [ ]:
# label
label_list = raw_data['label'].tolist()
print(len(label_list))

save_filename = 'data/{0}/label_pre_0723.txt'.format(folder)
print(save_filename)
fid = open(save_filename, "w+", encoding='UTF-8')
for data in label_pre:
    fid.write(data + '\n')
fid.close()

In [ ]:
# title
title_list = raw_data['title'].tolist()
print(len(title_list))
title_pre = pre_cor.handle_contents(title_list)

save_filename = 'data/{0}/title_pre_0723.txt'.format(folder)
print(save_filename)
fid = open(save_filename, "w+", encoding='UTF-8')
for data in title_pre:
    fid.write(data + '\n')
fid.close()

In [73]:
# content
content_list = raw_data['content'].tolist()
print(len(content_list))
content_pre = pre_cor.handle_contents(content_list)

save_filename = 'data/{0}/content_pre_0723.txt'.format(folder)
print(save_filename)
fid = open(save_filename, "w+", encoding='UTF-8')
for data in content_pre:
    fid.write(data + '\n')
fid.close()

6111
data/20180703/corpus_pre_cor_0703.txt


# 保存本文件

In [70]:
if 0:
    import datetime as dt
    
    def output_HTML(read_file, output_file):
        from nbconvert import HTMLExporter
        import codecs
        import nbformat
        exporter = HTMLExporter()
        # read_file is '.ipynb', output_file is '.html'
        output_notebook = nbformat.read(read_file, as_version=4)
        output, resources = exporter.from_notebook_node(output_notebook)
        codecs.open(output_file, 'w', encoding='utf-8').write(output)

    html_file_folder = 'html_files'
    if not os.path.exists(html_file_folder):
        os.makedirs(html_file_folder)

    today = dt.datetime.now().strftime('%Y%m%d')
    current_file = 'circ_cor_model_1_pre.ipynb'
    output_file = 'html_files\%s_%s.html'%(os.path.splitext(current_file)[0], today)
    output_HTML(current_file, output_file)